In [2]:
import pandas as pd
import folium
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Carregar o conjunto de dados
tabela = pd.read_csv("dataSet.csv", sep=';')

# Filtrar o DataFrame para incluir apenas os alunos que desistiram e têm até 20 anos de idade
alunos_desistentes_ate_20 = tabela[(tabela['SITUDESC'] == 'DESIST/DEIXOU FREQUE') & (tabela['IDADE'] <= 20)]

# Calcular o número de desistências por idade e sexo
desistencias_por_idade_sexo = alunos_desistentes_ate_20.groupby(['IDADE', 'SEXOALUNO']).size().unstack().reset_index()

# Configuração do Dash
app = dash.Dash(__name__)

# Layout do Dashboard
app.layout = html.Div([
    html.H1("Dashboard de Análise de Alunos - 2023"),

    # Gráfico: Distribuição de Idade dos Alunos
    html.Div([
        html.H2("Distribuição de Idade dos Alunos"),
        dcc.Graph(
            id='histograma-idade',
            figure=px.histogram(tabela, x='IDADE', nbins=20, title='Distribuição de Idade dos Alunos')
        )
    ]),

    # Gráfico: Distribuição de Alunos por Sexo
    html.Div([
        html.H2("Distribuição de Alunos por Sexo"),
        dcc.Graph(
            id='barra-sexo',
            figure=px.bar(x=tabela['SEXOALUNO'].value_counts().index, y=tabela['SEXOALUNO'].value_counts().values,
                          title='Distribuição de Alunos por Sexo', labels={'x': 'Sexo', 'y': 'Número de Alunos'})
        )
    ]),

    # Gráfico: Distribuição de Alunos por Série
    html.Div([
        html.H2("Distribuição de Alunos por Série"),
        dcc.Graph(
            id='barra-serie',
            figure=px.bar(x=tabela['SERIE'].value_counts().sort_index().index,
                          y=tabela['SERIE'].value_counts().sort_index().values,
                          title='Distribuição de Alunos por Série', labels={'x': 'Série', 'y': 'Número de Alunos'})
        )
    ]),

    # Gráfico: Desistências por Idade e Sexo (Até 20 anos)
    html.Div([
        html.H2("Desistências por Idade e Sexo (Até 20 anos)"),
        dcc.Graph(
            id='barra-desistencias-idade-sexo',
            figure=px.bar(desistencias_por_idade_sexo, x='IDADE', y=[col for col in desistencias_por_idade_sexo.columns if col != 'IDADE'],
                          title='Desistências por Idade e Sexo (Até 20 anos)', labels={'IDADE': 'Idade', 'value': 'Número de Desistências', 'variable': 'Sexo'},
                          barmode='group')
        )
    ]),

])

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
